In this notebook we read each consumer's csv file and create a dataframe with which to generate a single line of summary data

In [ ]:
import pandas as pd
import os

seasons = ["winter_wd", "winter_we", "summer_wd", "summer_we"]
# FIRST Setup a summary dataframe which we can write to a csv at the end.
# The summary df will include a row for each LCLid
# Each row will contain the LCLid in the first columnn then the following
# 24 columns will be the hourly mean() for the season.

# Create a list of strings containing the two-digit number from 00 to 23
# These will be our column names for the hourly data columns.
hourly_cols = [str(hour).zfill(2) for hour in range(24)]

# THEN loop through the seasons to create winter we/wd and summmer we/wd summary dataframes
for season in seasons:
    directory = "data_2013/" + season

    # Set up the summary df for each season
    summary_df = pd.DataFrame(columns=["LCLid"] + hourly_cols)

    # Now loop through the each seasonal folder and read each consumer's csv file
    # into a temporary dataframe and then calculate the mean KWH for each hour.
    # Then add the mean KWH values to the summary row.

    for filename in os.listdir(directory):

        if filename.startswith("MAC") and filename.endswith(".csv"):
            LCLid = filename.split(".")[0]
            print("Processing: ", season, LCLid)
            df = pd.read_csv(directory + "/" + filename, parse_dates=["HourlyDateTime"])

            df["Time"] = df["HourlyDateTime"].dt.time
            df.drop("HourlyDateTime", axis=1, inplace=True)

            # Now calculate the mean KWH for each hour into a list
            mean_kwh_list = list(df.groupby(["Time"])["KWH"].mean())

            # Now make a dictionary of column names and values by zipping the two lists
            mean_kwh = dict(zip(hourly_cols, mean_kwh_list))

            # Add a summary row to summary_df which contains "LCLid" and hourly mean values
            summary_row = {"LCLid": LCLid, **mean_kwh}

            # now add the summary_row to the summary_df using loc(len(df))
            summary_df.loc[len(summary_df)] = summary_row

    summary_df.to_csv(f"data_2013/summary/{season}.csv", index=False)

